# Train a PaddleOCR Text Detection Model on Colab

This notebook guides you through fine-tuning a **DBNet** text detection model on a custom dataset.

## 1. Setup Environment

In [ ]:
# Install PaddlePaddle GPU and PaddleOCR
!python -m pip install paddlepaddle-gpu==2.6.1 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install "paddleocr>=2.7.0"

In [ ]:
# Clone the PaddleOCR repository to access training scripts and configs
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
%cd PaddleOCR

## 2. Prepare Dataset

We'll use the ICDAR 2015 dataset as an example. We download it, unzip it, and format the labels.

In [ ]:
# Download and unzip the dataset
!wget https://paddleocr.bj.bcebos.com/dataset/rec_data/icdar_c4_train_imgs.zip -O train_data.zip
!unzip -q train_data.zip

# The default label file needs to be pointed to the correct image path
with open('icdar_c4_train_imgs/tra.txt', 'r') as f_in, open('train_label.txt', 'w') as f_out:
    for line in f_in:
        img_name, text = line.strip().split(',')
        f_out.write(f'icdar_c4_train_imgs/{img_name}\t{text}\n')

## 3. Configure Training

We'll use a lightweight MobileNetV3 DBNet configuration and modify it for our dataset.

In [ ]:
# Pre-download the backbone weights
!wget https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/MobileNetV3_large_x0_5_pretrained.pdparams -P ./pretrain_models/

## 4. Launch Training

We start the training process. This may take a while. We'll train for only 50 epochs for this demo.

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!python3 tools/train.py \
    -c configs/det/det_mv3_db.yml \
    -o Global.pretrain_weights=./pretrain_models/MobileNetV3_large_x0_5_pretrained \
       Global.save_model_dir=./output/det_db_icdar15 \
       Global.epoch_num=50 \
       Train.dataset.label_file_list=['./train_label.txt'] \
       Train.dataset.data_dir=./ \
       Eval.dataset.label_file_list=['./train_label.txt'] \
       Eval.dataset.data_dir=./ \
       Train.loader.batch_size_per_card=8

## 5. Test Inference
Let's test the trained model on a sample image.

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image

ocr = PaddleOCR(det_model_dir='./output/det_db_icdar15/best_accuracy', use_angle_cls=False, lang='en', use_gpu=True, show_log=False)

img_path = './doc/imgs/11.jpg'
result = ocr.ocr(img_path, det=True, rec=False, cls=False)

image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result[0]]
im_show = draw_ocr(image, boxes, font_path='./doc/fonts/latin.ttf')
Image.fromarray(im_show).save('result.jpg')
display(Image.open('result.jpg'))